In [60]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris

In [61]:
# url = 'https://gist.githubusercontent.com/curran/a08a1080b88344b0c8a7/raw/0e7a9b0a5d22642a06d3d5b9bcbad9890c8ee534/iris.csv'
# df = pd.read_csv(url)

In [62]:
# df.head()

In [63]:
iris = load_iris()
X = iris.data
y = iris.target

In [64]:
X = iris.data
y = iris.target

In [65]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [66]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [67]:
class IrisModel(nn.Module):
    def __init__ (self, NUM_FEATURES, NUM_CLASSES, NUM_HIDDEN):
        super().__init__()
        self.layer1 = nn.Linear(NUM_FEATURES, NUM_HIDDEN)
        # self.layer2 = nn.Linear(NUM_HIDDEN, NUM_CLASSES)
        self.layer2 = nn.Linear(NUM_HIDDEN, 4)
        self.layer3 = nn.Linear(4, NUM_CLASSES)
        self.log_softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = self.layer1(x)
        x = torch.sigmoid(x)
        x = self.layer2(x)
        x = torch.sigmoid(x)
        x = self.layer3(x)
        x = self.log_softmax(x)

        return x

In [68]:
# hyperparameters
NUM_FEATURES = X.shape[1]
NUM_CLASSES = len(iris.target_names)
NUM_HIDDEN = 6

NUM_FEATURES, NUM_CLASSES, NUM_HIDDEN

(4, 3, 6)

In [69]:
model = IrisModel(NUM_FEATURES, NUM_CLASSES, NUM_HIDDEN)

In [70]:
from torchsummary import summary

summary(model, input_size=(NUM_FEATURES,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                    [-1, 6]              30
            Linear-2                    [-1, 4]              28
            Linear-3                    [-1, 3]              15
        LogSoftmax-4                    [-1, 3]               0
Total params: 73
Trainable params: 73
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [71]:
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [72]:
EPOCHS = 150

X_train_tensor = torch.tensor(X_train)
y_train_tensor = torch.tensor(y_train)

for epoch in range(EPOCHS):
    optimizer.zero_grad()
    y_pred = model(X_train_tensor)
    l = loss(y_pred, y_train_tensor)
    l.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f'epoch {epoch}: {l.item()}')

epoch 0: 1.2049062252044678
epoch 10: 1.1127952337265015
epoch 20: 1.0722743272781372
epoch 30: 1.0329511165618896
epoch 40: 0.9576631784439087
epoch 50: 0.8416792154312134
epoch 60: 0.7154772281646729
epoch 70: 0.6196635365486145
epoch 80: 0.5615345239639282
epoch 90: 0.527442991733551
epoch 100: 0.5047621130943298
epoch 110: 0.48479291796684265
epoch 120: 0.4620514214038849
epoch 130: 0.43430107831954956
epoch 140: 0.40253421664237976


In [73]:
with torch.no_grad():
    y_pred = model(torch.tensor(X_test))
    correct = (torch.argmax(y_pred, dim=1) == torch.tensor(y_test)).sum().item()

    print(f'Accuracy: {correct/len(y_test)}')

Accuracy: 0.9666666666666667


In [74]:
# Ensure y_test is a tensor and on the same device as y_pred
y_test_tensor = torch.tensor(y_test, device=y_pred.device)
accuracy = accuracy_score(y_test_tensor.cpu(), torch.argmax(y_pred, dim=1).cpu())

In [75]:
accuracy

0.9666666666666667

In [76]:
torch.save(model.state_dict(), 'IrisModel.pth')

In [77]:
torch.save(model, 'IrisModel.h5')

In [78]:
load_model = torch.load('IrisModel.h5')
load_model.eval()

/var/folders/n_/tb_qk64d4q740jlrg8457tvc0000gn/T/ipykernel_1179/3404859138.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  load_model = torch.load('IrisModel.h5')


IrisModel(
  (layer1): Linear(in_features=4, out_features=6, bias=True)
  (layer2): Linear(in_features=6, out_features=4, bias=True)
  (layer3): Linear(in_features=4, out_features=3, bias=True)
  (log_softmax): LogSoftmax(dim=1)
)

In [83]:
# predict a new sample
sample = np.array([5.1, 3.5, 1.4, 0.2]).astype('float32')
sample_tensor = torch.tensor(sample).unsqueeze(0)  # Add batch dimension

with torch.no_grad():
    y_pred = load_model(sample_tensor)
    print(f'predicted class: {torch.argmax(y_pred).item()}')
    print(f'Predicted class: {iris.target_names[torch.argmax(y_pred).item()]}')

predicted class: 0
Predicted class: setosa
